<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [113]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [115]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [116]:
# текст запроса
query_3_1 = f'''select count(id) 
            from public.vacancies 
'''

In [117]:
# результат запроса
df_vacancies_count = pd.read_sql_query(query_3_1, connection)
display(df_vacancies_count)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [118]:
# текст запроса
query_3_2 = f'''select count(id) 
            from public.employers 
'''

In [119]:
# результат запроса
df_employers_count = pd.read_sql_query(query_3_2, connection)
display(df_employers_count)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [120]:
# текст запроса
query_3_3 = f'''select count(id) 
            from public.areas 
'''

In [121]:
# результат запроса
df_areas_count = pd.read_sql_query(query_3_3, connection)
display(df_areas_count)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [122]:
# текст запроса
query_3_4 = f'''select count(id) 
            from public.industries 
'''

In [123]:
# результат запроса
df_industries_count = pd.read_sql_query(query_3_4, connection)
display(df_industries_count)

,count
0,294


***

In [124]:
# Дополнительный расчет к юниту 3.

query_3_add = f'''select count(id) vacancies, count(distinct area_id) areas, count(distinct employer_id) employers
            from public.vacancies
'''

df_vacancies_details = pd.read_sql_query(query_3_add, connection)
display(df_vacancies_details)

,vacancies,areas,employers
0,49197,769,14906


Выводы по предварительному анализу данных:

В таблице вакансий имеется 49197 предложений от 14906 (из 23501) работодателей в 769 (из 1362) регионах. Т.о. не все работодатели представлены в таблице с вакансиями.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [125]:
# текст запроса
query_4_1 = f'''select a.name area, count(v.id) cnt
            from public.areas a
            join public.vacancies v on a.id=v.area_id
            group by a.name
            order by 2 desc
            limit 25
'''

In [126]:
# результат запроса
df_top5_areas = pd.read_sql_query(query_4_1, connection)
display(df_top5_areas)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [127]:
# текст запроса
query_4_2 = f'''select count(id) 
            from public.vacancies
            where salary_from is not null or salary_to is not null
'''

In [128]:
# результат запроса
df_filled_salary = pd.read_sql_query(query_4_2, connection)
display(df_filled_salary)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [129]:
# текст запроса
query_4_3 = f'''select round(avg(salary_from), 0), round(avg(salary_to), 0)
            from public.vacancies
'''

In [130]:
# результат запроса
df_avg_salary = pd.read_sql_query(query_4_3, connection)
display(df_avg_salary)

,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [131]:
# текст запроса
query_4_4 = f'''select schedule, employment, count(id) 
            from public.vacancies
            group by schedule, employment
            order by count(id) desc
'''

In [132]:
# результат запроса
df_schedule_employment = pd.read_sql_query(query_4_4, connection)
display(df_schedule_employment)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [133]:
# текст запроса
query_4_5 = f'''select experience, count(id) 
            from public.vacancies
            group by experience
            order by 2 asc
'''

In [134]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)
display(df_experience)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий:

Ожидаемо, что Москва, Санкт-петербург, столицы Беларуси и Казахстана, а также крупные города-милионники России являются лидерами по числу вакансий. 

В почти половине вакансий не указана хотя бы одна граница заработной платы, т.е. работодатели предпочитают отталкиваться от зарплатных ожиданий соискателей. Также возможная причина - конфиденциальность информации о зарплате, т.к. другие сотрудники могут смотреть вакансии своего работодателя.

Больше половины вакансий относятся к начинающим специалистам и меньше всего - к опытным специалистам с опытом работы более 6 лет. Это может говорить о том, что работодатель предпочитает вырастить специалиста внутри компании, чтобы он усвоил ее культуру и стиль работы.

Разница в верхней и нижней средней зарплате существенная. Вероятно окончательная зарплата сильно зависит от специализации, набота навыков и опыта соискателя.

Самая востребованная пара тип занятости/график работы у работодателей - полная занятость и полный день, доля почти 72%. Удаленную работу при полной занятости предлагают лишь около 16% работодателей. Таким образом, большинство работодателей предпочитают офисный формат работы на полный день, а удаленная работа не настолько популярна. Возможно, работодатели считают работу в офисе более эффективной или хотят иметь больше возможности контролировать сотрудников, а также поддерживать личную коммуникацию, котоая для многих более комфортна, чем общение в мессенджере или по электронной почте.


In [135]:
# выводы по детальному анализу вакансий


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [136]:
# текст запроса
query_5_1 = f'''select e.name, count(v.id) 
            from public.employers e
            join public.vacancies v on v.employer_id = e.id
            group by e.id, e.name
            order by 2 desc
            limit 20
'''

In [137]:
# результат запроса
df_employers_vacancies = pd.read_sql_query(query_5_1, connection)
display(df_employers_vacancies)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [138]:
# текст запроса
query_5_2 = f'''select ar.name, count(distinct em.id) number_of_employers, count(v.id) number_of_vacancies
            from public.areas ar
            left join public.employers em on ar.id = em.area
            left join public.vacancies v on (v.area_id = ar.id) and (v.employer_id = em.id)
            group by ar.name
            having count(v.id) = 0
            order by 3 asc, 2 desc
            limit 1
'''
query_5_2_all = f'''select ar.name, count(distinct em.id) number_of_employers, count(v.id) number_of_vacancies
            from public.areas ar
            left join public.employers em on ar.id = em.area
            left join public.vacancies v on (v.area_id = ar.id) and (v.employer_id = em.id)
            group by ar.name
            order by 3 desc, 2 desc
            limit 25
'''


In [139]:
# результат запроса
df_no_vacancies = pd.read_sql_query(query_5_2, connection)
display(df_no_vacancies)

df_vacancies_by_areas = pd.read_sql_query(query_5_2_all, connection)
display(df_vacancies_by_areas)

,name,number_of_employers,number_of_vacancies
0,Россия,410,0


,name,number_of_employers,number_of_vacancies
0,Москва,5864,4567
1,Минск,1115,1858
2,Санкт-Петербург,2217,1811
3,Алматы,721,1069
4,Екатеринбург,609,960
5,Новосибирск,573,946
6,Казань,480,719
7,Ташкент,326,655
8,Нижний Новгород,426,633
9,Краснодар,409,565


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [140]:
# текст запроса
query_5_3 = f'''select em.name, count(distinct v.area_id) 
            from public.employers em
            join public.vacancies v on v.employer_id = em.id
            group by em.name
            order by 2 desc
            limit 20
'''


In [141]:
# результат запроса
df_area_quantity = pd.read_sql_query(query_5_3, connection)
display(df_area_quantity)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [142]:
# текст запроса
query_5_4 = f'''select e.name
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            group by e.id
            having count(ei.industry_id) = 0            
'''


In [143]:
# результат запроса
df_blank_industry = pd.read_sql_query(query_5_4, connection)
print(df_blank_industry.shape[0])
display(df_blank_industry)

8419


,name
0,Северо-Западный Информационно-Мониторинговый Ц...
1,АВГ Инженерные Системы
2,Лидстех
3,Евроклимат Тольятти
4,Finstar Financial Group
...,...
8414,ОГКУ ЦЗН города Бодайбо
8415,ПП «Южуралкомсервис»
8416,Whitecliff Investment Management
8417,TransPerfect


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [144]:
# текст запроса
query_5_5 = f'''select e.name 
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            group by e.id
            having count(ei.industry_id) = 4
            order by 1 asc
            offset 2
            limit 1            
'''

In [145]:
# результат запроса
df_four_industries = pd.read_sql_query(query_5_5, connection)
display(df_four_industries)

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [146]:
# текст запроса
query_5_6 = f'''select count(distinct e.id)
            from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            join public.industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'           
'''

In [147]:
# результат запроса
df_sw_dev_industry = pd.read_sql_query(query_5_6, connection)
display(df_sw_dev_industry)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [148]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
# Находим первую таблицу на сайте, которая содержит данные по городам-миллионникам.
table = page.find('table')
print(table.text)
cities = []
# Организуем цикл для прохода по строкам таблицы, начиная со второй, и по второму столбцу, где указаны города-миллионники.
for i in table.find_all('tr')[1:]:
    rows = i.find_all('td')
    city = rows[1].text
    cities.append(city)
print(cities)
# Кортеж нужен для подстановки в f-строку в тексте запроса.
cities = tuple(cities)


№ГородНаселение,  тыс. чел.  (на 1 января 2024)[2]Население,  тыс. чел. (перепись-2021)[3]Прирост относительно  переписи-2010Население,  тыс. чел.(перепись-2010)[4]1Москва13 15013 01013,1 %11 5042Санкт-Петербург5 5985 60214,8 %4 8803Новосибирск1 6341 63410,9 %1 4744Екатеринбург1 5361 54414,4 %1 3505Казань1 3191 30914,4 %1 1446Красноярск1 2051 18821,6 %9777Нижний Новгород1 2051 2260,2−2,0 %1 2518Челябинск1 1771 1905,3 %1 1309Уфа1 1631 1457,8 %1 06210Самара1 1591 1730,3 %1 16911Ростов-на-Дону1 1401 1424,9 %1 08912Краснодар1 1391 09943,8 %76413Омск1 1041 1260,1−3,4 %1 16614Воронеж1 0461 05818,9 %89015Пермь1 0271 0344,3 %99116Волгоград1 0191 0280,7 %1 021
['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [149]:
# текст запроса
query_5_7 = f'''select ar.name, count(v.id)
            from public.areas ar
            left join public.vacancies v on v.area_id = ar.id
            join public.employers em on em.id = v.employer_id
            where (em.name = 'Яндекс') and (ar.name in {cities})
            group by ar.name
            union
            select 'TOTAL', count(v.id)
            from public.areas ar
            left join public.vacancies v on v.area_id = ar.id
            join public.employers em on em.id = v.employer_id
            where (em.name = 'Яндекс') and (ar.name in {cities})
            order by 2 asc
'''

In [150]:
# результат запроса
df_yandex_vacancies = pd.read_sql_query(query_5_7, connection)
display(df_yandex_vacancies)

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Выводы по анализу работодателей:

В числе компаний, имеющих наибольшее количество вакансий - ИТ компании, банки (финтех России хорошо развит), операторы связи и добывающий сектор. Лидер - компания Яндекс. Все эти отрасли активно развиваются и применяют ИТ технологии.

Компания Яндекс также является лидером по числу регионов, в которых она имеет вакансии. Это говорит о том, что Яндекс играет большую роль в экономике России и ее технологическом развитии.

7,2% работодателей занимаются разработкой программного обеспечения. Это связано с развитием ИТ сектора. Государство активно поддерживает эту отрасль, в том числе налоговыми льготами. Например, пониженными налоговыми ставками.

У многих (8419 из 23501)  работодателей не указана сфера деятельности. Можно дополнительно исследовать таких работодателей на предмет наличия у них существенного количества вакансий. Возможно многие из них те, что есть в базе, но у нх отсутствуют вакансии.


In [151]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [152]:
# текст запроса
query_6_1 = f'''select count(id)
            from public.vacancies
            where (lower(name) like '%data%') or (lower(name) like '%данн%')
'''

In [153]:
# результат запроса
df_data_vacancies = pd.read_sql_query(query_6_1, connection)
display(df_data_vacancies)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [154]:
# текст запроса
query_6_2 = f'''select count(id)
            from public.vacancies
            where (lower(name) not like '%html%') and 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            (lower(name) like '%ml%') or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and 
            ((experience = 'Нет опыта') or (employment = 'Стажировка') or (lower(name) like '%junior%'))
'''

In [155]:
# результат запроса
df_junior_ds = pd.read_sql_query(query_6_2, connection)
display(df_junior_ds)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [156]:
# текст запроса
query_6_3 = f'''select count(id)
            from public.vacancies
            where (lower(name) not like '%html%') and 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            (lower(name) like '%ml%') or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and 
            ((lower(key_skills) like '%sql%') or (lower(key_skills) like '%postgres%'))
'''

In [157]:
# результат запроса
df_sql_postges = pd.read_sql_query(query_6_3, connection)
display(df_sql_postges)

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [158]:
# текст запроса
query_6_4 = f'''select count(id)
            from public.vacancies
            where (lower(name) not like '%html%') and 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            (lower(name) like '%ml%') or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and 
            ((lower(key_skills) like '%python%'))
'''

In [159]:
# результат запроса
df_python = pd.read_sql_query(query_6_4, connection)
display(df_python)

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [160]:
# текст запроса
query_6_5 = f'''select round(avg(length(key_skills) - length(replace(key_skills, '\t', ''))+1), 2)
            from public.vacancies
            where  
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and
            key_skills is not null
'''

In [161]:
# результат запроса
df_number_key_skills = pd.read_sql_query(query_6_5, connection)
display(df_number_key_skills)

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [162]:
# текст запроса
query_6_6 = f'''select experience, count(id), round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2), 0)
            from public.vacancies
            where  
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and
            (salary_from is not null or salary_to is not null)
            group by experience
'''

In [163]:
# результат запроса
df_ds_salary = pd.read_sql_query(query_6_6, connection)
display(df_ds_salary)

,experience,count,round
0,Нет опыта,7,74643.0
1,От 1 года до 3 лет,28,139675.0
2,От 3 до 6 лет,27,243115.0


***

Выводы по предметному анализу:

Анализ зарплат дата-сайентистов показывает, что различие вознаграждения между категориями опыта значительное (оно почти удваивается при переходе в следующую категорию). Это может говорить о том, что спрос на опытных дата-сайентистов очень высокий.

В среднем в вакансиях на позицию дата-сайентист указано 6 ключевых навыков. В 357 вакансиях из 480 (см. дополнительное исследование 1) требуется владение Python и в 229 - SQL. Таким образом, это очень востребованные навыки.

Треть вакансий, имеющих отношение к данным (27%, см. также дополнительное исследование 1) - дата-сайентисты. Т.е. это довольно востребованное направление среди таких специальностей.

Junior специалистам по дата-сайенс предлагается 51 вакансия, что относительно немного. Но это обычная ситуация, характерная для многих сфер. Начинающему специалисту часто непросто найти работу, но, набравшись опыта, они станвятся востребованными.


In [164]:
# выводы по предметному анализу

# Общий вывод по проекту

In [165]:
# Дополнительное исследование 1. Вакансии по DS в разрезе опыта работы.

query_add_1 = f'''select experience, count(id)
            from public.vacancies
            where 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%'))
            group by experience
            union
            select 'TOTAL', count(id)
            from public.vacancies
            where 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%'))
            order by 2 desc
'''
df_ds = pd.read_sql_query(query_add_1, connection)
display(df_ds)


,experience,count
0,TOTAL,480
1,От 1 года до 3 лет,223
2,От 3 до 6 лет,195
3,Более 6 лет,34
4,Нет опыта,28


Исходя из данных по количеству вакансий по дата-сайеннс в разрезе опыта работы, начанающему специалисту будет не просто найти работу. Однако имея опыт работы от года до трёх лет шансы существенно повышаются, т.к. это самая востребованная работодателями категория специалистов.

In [166]:
# Дополнительное исследование 2. Нижняя и верхняя гранца зарплаты специалистов по DS в разрезе опыта работы.

query_add_2 = f'''select experience, round(avg(salary_from), 1) DS_salary_from, round(avg(salary_to), 1) DS_salary_to
            from public.vacancies
            where 
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and
            (salary_from is not null or salary_to is not null)
            group by experience
'''
query_add_2_1 = f'''select experience, round(avg(salary_from), 1) salary_from, round(avg(salary_to), 1) salary_to
            from public.vacancies
            where 
            (salary_from is not null or salary_to is not null)
            group by experience
'''

df_avg_ds_salary = pd.read_sql_query(query_add_2, connection)
display(df_avg_ds_salary)

df_avg_salary = pd.read_sql_query(query_add_2_1, connection)
display(df_avg_salary)

,experience,ds_salary_from,ds_salary_to
0,Нет опыта,61000.0,97000.0
1,От 1 года до 3 лет,121272.0,175672.8
2,От 3 до 6 лет,198948.6,290589.4


,experience,salary_from,salary_to
0,Более 6 лет,150155.5,197819.2
1,От 3 до 6 лет,112544.1,171974.0
2,Нет опыта,34698.7,56990.5
3,От 1 года до 3 лет,64456.4,97435.9


Широкий разброс нижней и верхней границы предлагаемой зарплаты характерен для всех категорий опыта работы. Вероятно, фактическая зарплата зависит от возможностей компании-работодателя, набора навыков соискателя, опыта работы в конкретном направлении, которое актуально для работодателя.

Также можно заметить, что средняя нижняя и верхняя границы предлагаемой заработной платы для дата-сайентистов значительно выше средних по всем вакансиям.

In [167]:
# Дополнительное исследование 3. Ключевые навыки для DS.

query_add_3 = f'''select key_skills
            from public.vacancies
            where  
            ((lower(name) like '%data scientist%') or 
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or 
            ((name like '%ML%') and (lower(name) not like '%html%')) or 
            (lower(name) like '%machine learning%') or 
            (lower(name) like '%машинн%обучен%')) and
            key_skills is not null
'''

df_number_key_skills = pd.read_sql_query(query_add_3, connection)

# Органзуем цикл для получения списка ключевых навыков для дата-сайентистов.
key_skills_list = []
for elem in df_number_key_skills['key_skills']:
    key_skills_list.extend(elem.split('\t'))

# Получаем десятку самых востребованных навыков.
df_ds_skills = pd.DataFrame(data=key_skills_list)
display(df_ds_skills.value_counts().head(10))


0                        
Python                       348
SQL                          191
Machine Learning             114
Git                           66
Математическая статистика     62
Data Analysis                 54
Pandas                        52
Data Science                  52
ML                            49
Английский язык               48
Name: count, dtype: int64

In [168]:
connection.close()

Кроме Python и SQL среди самых вотребованных навыков для специалистов по DS также выделяются машинное обучение, Git, математическая статистика, анализ данных и владение английским языком.

*** Общие выводы ***

Примечательно, что работодателем с самым большим количеством вакансий является ИТ компания Яндекс с существенным отрывом от второго по количеству вакансий работодателя. Причем она также является лидером по количеству регионов. 

Как показывает практика, автоматизация процессов в различных областях не приводит к уменьшению потребности в работниках. Просто происходит перераспределение по профессиям. В частности, появляется больше ваканий, так или иначе связанных с разработкой ИТ продуктов.

Около 1% вакансий связаны с дата-сайенс. При этом уровень предлагаемых зарплат заметно превышает средние значения.

В зависимости от цели можно продолжить исследование в разных направлениях.

Из юнита 5, задание 2 видно, что в одних регионах работодателей больше, чем вакансий, а в других - наоборот. Можно изучить потребность в кадрах работодателей в зависимости от региона от индустрии. Отсутствие индустрии у многих работодателей можно заполнить данными по основному виду деятельности (ОКВЭД) из различных баз данных (например, СПАРК). Но предварительно нужно выяснить, насколько существенно количество вакансий работодателей, не указавших сферу своей деятельности.

Можно также исследовать предлагаемые зарплаты в зависимости от вида деятельности работодателя.

